In [15]:
import os
import argparse
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from dataflow import LcqmcDataset
from model import RE2
from NNUtils.torchwu.cuda import try_gpu
from util import train, validate

In [16]:
# -------------------- Prepare for params ------------------- #
device = try_gpu()
epochs = 5
max_length = 30
batch_size = 1
lr = 1e-4
max_grad_norm=10.0
patience=5

In [17]:
# -------------------- Data loader ------------------- #
train_data = LcqmcDataset('train_char.csv', max_len=max_length,data_size=100)
dl_train = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_data = LcqmcDataset('test_char.csv', max_len=max_length)
dl_test = DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [21]:
# -------------------- Model definition ------------------- #
class ModelArgs:
    def __init__(self):
        self.dropout=0.2
        self.embedding_dim=300
        self.hidden_size=150
        self.kernel_sizes=[3]
        self.blocks=2
        self.fix_embeddings=True
        self.encoder='cnn'
        self.enc_layers=2
        self.alignment='linear'
        self.fusion='full'
        self.connection='aug'
        self.prediction='full'
        self.num_classes=2

args = ModelArgs()
vocab = train_data.get_vocab()
embeddings = np.random.randn(len(vocab),30)
model = RE2(args, embeddings, device=device).to(device)


In [22]:
# -------------------- Preparation for training  ------------------- #
criterion = nn.CrossEntropyLoss()
# 过滤出需要梯度更新的参数
parameters = filter(lambda p: p.requires_grad, model.parameters())
# optimizer = optim.Adadelta(parameters, params["LEARNING_RATE"])
optimizer = torch.optim.Adam(parameters, lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="max",
                                                       factor=0.85, patience=0)

In [23]:
# -------------------- Training step ------------------- #

for epoch in range(3):
    epoch_time, epoch_loss, epoch_accuracy = train(model, dl_train, optimizer,criterion, epoch, max_grad_norm)

    scheduler.step(epoch_accuracy)
    print("-> Training time: {:.4f}s, loss = {:.4f}, accuracy: {:.4f}%"
          .format(epoch_time, epoch_loss, (epoch_accuracy*100)))

  0%|          | 0/100 [00:00<?, ?it/s]


RuntimeError: Expected 3-dimensional input for 3-dimensional weight [150, 300, 3], but got 4-dimensional input of size [1, 30, 100, 30] instead